In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from utils.imports import *
from input_fn import input_fn
from cvt2tfrecord import fn_record_to_count
from utils.clr import LRFinder

# from tensorflow.keras.layers import *
# import tensorflow.keras as keras
# from tensorflow.keras import backend as K
# from tensorflow.keras.models import Model

Using TensorFlow backend.


In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" # so the IDs match nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1" # "0, 1" for multiple

In [19]:
# from utils.transforms import *
PATH = Path('../data')
sz = 128
# sz = 256
nt = 10
bs = 16
MODEL_VERSION = 'prednet_' + str(sz) + '_1'
MODEL_PATH = PATH/'models'

num_gpus = 1

# class Slice(Transform):
#     """ Return a slice of the images
    
#     Arguments:
#     The same as the built-in function slice
#     """
#     def __init__(self, *args, **kwargs):
#         self.slice = slice(*args, **kwargs)
#         super().__init__(TfmType.NO)
        
#     def do_transform(self, x, is_y):
#         return x[self.slice]

# aug_tfms = [Slice(nt)]
aug_tfms = []

In [5]:
from models.prednet_refactored import PredNetCell, PredNet

# n_channels, im_height, im_width = (3, 128, 160)
n_channels, im_height, im_width = (1, sz, sz)
input_shape = (im_height, im_width, n_channels)
stack_sizes = (n_channels, 48, 96, 192)
R_stack_sizes = stack_sizes
A_filt_sizes = (3, 3, 3)
Ahat_filt_sizes = (3, 3, 3, 3)
R_filt_sizes = (3, 3, 3, 3)

layer_loss_weights = np.array([1., 0., 0., 0.])  # weighting for each layer in final loss; "L_0" model:  [1, 0, 0, 0], "L_all": [1, 0.1, 0.1, 0.1]
layer_loss_weights = np.expand_dims(layer_loss_weights, 1)
time_loss_weights = 1./ (nt - 1) * np.ones((nt,1))  # equally weight all timesteps except the first
time_loss_weights[0] = 0

prednet_cell = PredNetCell(stack_sizes=stack_sizes,
                    R_stack_sizes=R_stack_sizes,
                    A_filt_sizes=A_filt_sizes,
                    Ahat_filt_sizes=Ahat_filt_sizes,
                    R_filt_sizes=R_filt_sizes)

prednet = PredNet(prednet_cell)

In [6]:
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(allow_soft_placement=True,)
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
# set_session(sess)  # set this TensorFlow session as the default session for Keras

In [7]:
from tensorflow.keras.layers import TimeDistributed, Flatten, Dense
from tensorflow.keras.models import Model

inputs = tf.keras.Input(shape=(nt,) + input_shape)
errors = prednet(inputs)  # errors will be (batch_size, nt, nb_layers) 
errors_by_time = TimeDistributed(Dense(1, trainable=False), weights=[layer_loss_weights, np.zeros(1)], trainable=False)(errors)  # calculate weighted error by layer
errors_by_time = Flatten()(errors_by_time)  # will be (batch_size, nt)
final_errors = Dense(1, weights=[time_loss_weights, np.zeros(1)], trainable=False)(errors_by_time)  # weight errors by time
model = Model(inputs=inputs, outputs=final_errors)
# model = tf.keras.utils.multi_gpu_model(model, gpus=2)
model.compile(loss='mean_absolute_error', optimizer='adam')
# model.compile(loss='mean_absolute_error', optimizer=tf.train.AdamOptimizer())

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 128, 128, 1)   0         
_________________________________________________________________
pred_net (PredNet)           (None, 10, 4)             6909818   
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 1)             5         
_________________________________________________________________
flatten (Flatten)            (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 6,909,834
Trainable params: 6,909,818
Non-trainable params: 16
_________________________________________________________________


In [20]:
fns = ['../data/tfrecords/train_1_contiguous_10']
x, y, num_iterations = input_fn(bs, sz, nt, aug_tfms, fns, 'stat.csv', ',', num_parallel_calls=32)
# num_iterations is pre-batch iterations

In [10]:
# sess = tf.InteractiveSession()
# x.eval().shape

In [11]:
path_checkpoints = MODEL_PATH/'checkpoints'
if not path_checkpoints.exists():
    path_checkpoints.mkdir(parents=True)
model_callbacks = [
    # TODO: save info of prednet cell
    tf.keras.callbacks.TensorBoard(write_grads=True, write_images=True),
#     keras.callbacks.History(),
#     tf.keras.callbacks.ModelCheckpoint(str(path_checkpoints/('weights.' + MODEL_VERSION + '.{epoch:02d}-{val_loss:.2f}.hdf5'))),
    tf.keras.callbacks.ModelCheckpoint(str(path_checkpoints/('weights.' + MODEL_VERSION + '.{epoch:02d}.hdf5'))),
#     keras.callbacks.EarlyStopping()
]
minimum_lr = 1e-3
maximum_lr = 1
lr_callbacks = [LRFinder(num_iterations, bs,
                        minimum_lr, maximum_lr,
                       # validation_data=(X_val, Y_val),
                       lr_scale='exp', save_dir=str(path_checkpoints/'lrs'))]
# callbacks = model_callbacks + lr_callbacks
callbacks = model_callbacks

In [21]:
epochs = 1
total_iterations = epochs * num_iterations
model.fit(x, y, steps_per_epoch=num_iterations, callbacks=callbacks)

Epoch 1/1
    62/300000 [..............................] - ETA: 92:42:19 - loss: 0.0570

KeyboardInterrupt: 

In [23]:
sess = tf.InteractiveSession()
x.eval().shape

(16, 10, 128, 128, 1)

In [24]:
model.predict(x.eval(),)

FailedPreconditionError: Error while reading resource variable pred_net/Variable from Container: localhost. This could mean that the variable was uninitialized. Not found: Container localhost does not exist. (Could not find resource: localhost/pred_net/Variable)
	 [[{{node pred_net/MatMul/ReadVariableOp}} = ReadVariableOp[dtype=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](pred_net/Variable)]]
	 [[{{node dense_1/BiasAdd/_25}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_971_dense_1/BiasAdd", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]